In [4]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
D=28*28
n=2
C=1
classes=10

In [ ]:
class TransposeLinear(nn.Module):
    def __init__(self,linearlayer,bias=True):
        '''
        This function takes a linear layer and
        transposes it through weight sharing techniques 
        bias term = True means we will create a new bias for the transposed layer
        '''
        super().__init__()
        self.weight = linearlayer.weight

        if bias:
            self.bias = nn.Parameter(torch.Tensor(linearlayer.weight.shape[1]))
        else:
            self.register_parameter('bias',None)


    def forward(self,x:torch.Tensor):
        return F.linear(x,self.weight.t(),self.bias)